<a href="https://colab.research.google.com/github/DarekGit/FACES_DNN/blob/master/notebooks/05_02_MOBILENETV2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

---

[Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb) | [1. Wstęp](01_00_Wstep.ipynb) | [2. Metryki oceny detekcji](02_00_Miary.ipynb) | [3. Bazy danych](03_00_Datasety.ipynb) | [4. Przegląd metod detekcji](04_00_Modele.ipynb) | [5. Detekcja twarzy z wykorzystaniem wybranych architektur GSN](05_00_Modyfikacje.ipynb) | [6. Porównanie modeli](06_00_Porownanie.ipynb) | [7. Eksport modelu](07_00_Eksport_modelu.ipynb) | [8. Podsumowanie i wnioski](08_00_Podsumowanie.ipynb) | [Bibliografia](Bibliografia.ipynb)


---

<br>



< [5.1. Detectron2 Faster R-CNN z FPN Resnet50](05_01_DETECTRON2.ipynb) | [5.2. MOBILENETV2](05_02_MOBILENETV2.ipynb) | [5.3. Techniki szybkiego i stabilnego uczenia GSN.](05_03_FrozenBN_Mish.ipynb) >

## 5.2. MobileNet V2

W 2017 roku Zespół Google z dużym sukcesem wprowadził MobileNet V1 do klasyfikacji obrazów, jak i ekstracji cech w większych sieciach neuronowych.
Już rok później w 2018 zespół AI Google powiadomił o wprowadzeniu MobileNet version 2. <a href="https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Bibliografia.ipynb">[24]</a>

Na bazie V1 udało się stworzyć jeszcze bardziej efektywną i wydajną wersję, **do zastosowania na urządzeniach mobilnych.**


### Główne cechy MobileNet V1

Podstawowym pomysłem w V1 było wykorzystanie sperowalnych sieci konwolucyjnych w celu obniżenia zapotrzebowania na moc obliczeniową. Standardowe sieci konwolucyjne wymagają wykonania dużej liczby obliczeń, których ilość jest proporcjonalna do iloczynu ilości kanałów wejściowych, ilości kanałów wyjściowych, wielkości danych wejściowych i wielkości kernela. Zastosowanie konwolucji separowalnych <a href="https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Bibliografia.ipynb">[25]</a> po kanałach, obniża zapotrzebowanie na moc obliczeniową kilkukrotnie.

Podstawowa warstwa konwolucyjna została podzielona na dwa elementy: pierwszy to warstwa konwolucji separowanej po kanałach (depthwise convolution), po której mamy warstwę konwolucji 1×1 (pointwise) łączącej cechy z kanałów:<br><br>

<div align="center">
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_SeparableConv.png?raw=1" alt="Separable Convolution" width="300" >

</div>

<br>Razem obie warstwy tworzą blok konwolucyjny, który wykonuje podobne operacje do standardowej sieci konwolucyjnej, ale znacznie szybciej. (podobne rozwiązania stosujemy w sieciach typu Inception).
<br>Architektura Mobilenet V1 składa się z standardowej warstwy konwolucyjnej 3x3 a następnie z trzynastu bloków opisanych powyżej.

Pomiędzy warstwami konwolucji separowalnych nie stosujemy warstw Pool, zamiast tego wybrane warstwy konwolucji mają stride równe 2 w celu redukcji rozmiaru danych. Przy zastosowaniu redukcji rozmiaru danych, jednocześnie dublowana jest ilość kanałów wyjściowych w warstwie konwolucji 1x1 (pointwise). Przykładowo dla obrazu wejściowego o rozmiarze 224×224×3 na wyjściu sieci otrzymujemy mapę cech o rozmiarze 7×7×1024 na wyjściu.

Tak jak w większości współczesnych architektur, po warstwach konwolucyjnych stosujemy batch normalization, a jako funkcję aktywacji **ReLU6**. ReLU6 ma dodatkowe ograniczenie aby uniknąć zbyt dużych wartości:

<pre><code class="language-python">y = min(max(0, x), 6)
</code></pre>

Autorzy MobileNet w dokumentacji wskazują, iż sieć z ReLU6 jest bardziej odporna niż ReLU dla obliczeń z małą precyzją (16-bit float np dla iOS). 

Kształt ReLU6 ma zbliżone cechy do funkcji sigmoid:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_ReLU6.png?raw=1" alt="ReLU6" width="600" ><br>
</div>


<br>Sieć MobileNet w zastosowaniach do klasyfikacji kończy się blokiem składającym się z warstwy Average Pooling, FC lub alternatywnie Convolution 1x1 oraz Softmax.

<br>Sieć Mobilenet została przygotowana jako rodzina sieci. Mamy kilka hiperparametrów umożliwiających stosowanie różnych wersji sieci Mobilenet.
<br>Najważniejszym hiperparametrem jest  <strong>depth multiplier</strong>, czasami nazywany "width multiplier". Zmienia on ilość kanałów w każdej warstwie. 
<br>Stosując depth multiplier równy 0.5 obniżamy ilość kanałów, a tym samym ilość koniecznych obliczeń czterokrotnie oraz ilość parametrów sieci trzykrotnie, dzięki czemu sieć jest szybsza, ale  mniej dokładna.

<br>Dzięki zastosowaniu bloków z konwolucją separowalną MobileNet potrzebuje 9-krotnie mniej mocy obliczeniowej do porównywalnych sieci neuronowych z tą samą dokładnością klasyfikacji. Taki typ sieci może być stosowany z powodzeniem w czasie rzeczywistym do 200 warstw na iPhone 6s.


### MobileNet V2

MobileNet V2 <a href="https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Bibliografia.ipynb">[26]</a> bazuje na V1 i używa separowalnych warstw konwolucyjnych, a jego podstawowy blok wygląda następująco:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_ResidualBlock.png?raw=1" alt="bottleneck" width="300" ><br>
</div>

<br>W V2 mamy trzy warstwy konwolucyjne w bloku, dwie ostatnie są zgodne z wersją V1, ale konwolucja 1x1 ma inne zadanie. W wersji V1 zadaniem konwolucji 1x1 (pointwise) było utrzymanie lub zdublowanie ilości kanałów. W przypadku wersji V2 jej zadaniem jest zmniejszenie ilości kanałów. Dlatego nazywana jest **projection layer**, odwzorowuje większą ilość kanałów wejściowych na tensor z mniejszą ilością kanałów.
<br>Przykładowo warstwa Depthwise może pracować ze 144 kanałami, które są ograniczane do tylko 24 w warstwie Pointwise. Czasami warstwę Pointwise nazywa się **bottlnect layer** poniewż zawęża ilość danych przepływających przez sieć.

<br> Pierwsza warstwa w bloku spełnia nowe zadanie. Jest to również warstwa konwolucyjna 1x1, a jej zadaniem jest rozszerzenie ilości kanałów danych przed warstwą DepthWise. Nazywa się ją **expansion layer** i zawsze zwiększa ilość kanałów (odwrotnie niż **projection layer**).
Współczynnik ekspansji podawany jest jako hiperparametr i można go zmieniać. Domyślnie przyjmowany jest jako 6 (**expansion factor**).
<br>Przykładowo dla 24 kanałów wejściowych do bloku, Expansion Layer powiększa ilość kanałów do 144, następnie wykonywana jest konwolucja separowalna (Depthwise) na 144 kanałach i ostatecznie Projection Layer redukuje ilość kanałów do 24.

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_Expand.png?raw=1" alt="Expansion and projection" width="600" ><br>
</div>

<br>Tak więc wejście i wyjście ma mniejszy rozmiar, podczas gdy rozmiar wewnątrz bloku jest większy.
<br><br>Drugą nową cechą MobileNetV2 jest **residual connection**. Działa ono podobnie do ResNet i pomaga w utrzymaniu przepływu gradientu w sieci. Residual connection używane jest tylko i wyłącznie w przypadku kiedy ilość kanałów wejściowych jest równa ilości kanałów wyjściowych dla bloku.

<br>Jak w innych sieciach konwolucyjnych każda warstwa ma batch normalization i funkcję aktywacji (ReLU6), oprócz warstwy wyjściowej. Autorzy MobileNet w dokumentacji wskazują, iż warstwa nieliniowa na wyjściu dla małych rozmiarów danych wyjściowych pogarsza wyniki pracy sieci.

<br> Pełna architektura MobileNetV2 składa się ze standardowej warstwy konwolucyjnej 3x3 z 32 kanałami, następnie 17  bloków, zakończonych standardową warstwą konwolucyjną 1x1, Average Pool i warstwą klasyfikacyjną.
<br>
<br>


**Cel zmian**

<br>Pomysł dotyczący V1 opierał się na zmianie złożonej warstwy konwolucyjnej na mniej wymagająca warstwę separowalną **DeepWise**, co okazało sie dużym sukcesem. 
<br>Główną zmianą w wersji V2 jest zastosowanie **residual connection** oraz **expand/projection layers**, co umożliwia zachowanie małego rozmiaru na wejściu i wyjściu bloków:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2%20_LowDim.png?raw=1" alt="dimensions between the blocks" width="600" ><br>
</div>

<br> Rozmiar tensora wyjściowego dla wersji V1 (7×7×1024) jest większy od V2. Dzięki zachowaniu małego rozmiaru tensora uzyskano redukcją koniecznej liczby obliczeń dla sieci przy zachowaniu zadowalającej jakości klasyfikacji sieci.
<br>Podaje się porównanie, że mały rozmiar danych pomiędzy blokami odzwierciedla  skompresowane dane rzeczywiste, które są dekompresowane wewnątrz bloków:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_Compression.png?raw=1" alt="Decompression and compression " width="600" ><br>
</div>

<br> Expansion Layer działa jako dekompresor, który odtwarza dane rzeczywiste do przetwarzania wewnątrz bloku, a które są następnie kompresowane w Projection Layer w celu zmniejszenia rozmiaru danych na wyjściu bloku. Parametry całego procesu w tym kompresji i dekompresji podlegają optymalizacji w trakcie uczenia sieci.


### Porównanie wersji MobileNet.

<br>Poniższa tabela zawiera porównanie MobileNet V1 i V2, wymaganą ilość obliczeń oraz rozmiar modelu - ilość parametrów:
<br>

<table>
<thead>
<tr>
<th>Version</th>
<th>MACs (millions)</th>
<th>Parameters (millions)</th>
</tr>
</thead>

<tbody>
<tr>
<td>MobileNet V1</td>
<td>569</td>
<td>4.24</td>
</tr>

<tr>
<td>MobileNet V2</td>
<td>300</td>
<td>3.47</td>
</tr>
</tbody>
</table>

<br>Do porównania wykorzystano dane z <a href="https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet_v1.md">Github MNV1</a> oraz  <a href="https://github.com/tensorflow/models/tree/master/research/slim/nets/mobilenet">Github MNV2</a>. Modele w wersji z depth multiplier równe 1.0. Dla podanej tabeli, mniejsze liczby oznaczają lepszy wynik.

"MACs - multiply-accumulate operations" określa ile obliczeń jest wymaganych dla wykonania operacji dla sieci na pojedyńczym obrazie RGB o wymiarach 224x224.


Przewagi V2: 
1. Prawie dwa razy mniejszą ilość obliczeń,
1. Mniej parametrów (80%) dzięki czemu szybciej ładuje je z pamięci, co jest istotne dla urządzeń mobilnych, gdzie dostęp do pamięci jest znacznie wolniejszy niż obliczenia.

<br>Poniższa tabela pokazuje maksymalny FPS (frames-per-second) dla różnych urządzeń dla wersji modeli jak wyżej:</p>

<table>
<thead>
<tr>
<th>Version</th>
<th>iPhone 7</th>
<th>iPhone X</th>
<th>iPad Pro 10.5</th>
</tr>
</thead>

<tbody>
<tr>
<td>MobileNet V1</td>
<td>118</td>
<td>162</td>
<td>204</td>
</tr>

<tr>
<td>MobileNet V2</td>
<td>145</td>
<td>233</td>
<td>220</td>
</tr>
</tbody>
</table>

<br> Wyniki uzyskano przy zastosowaniu optymalizacji polegającej na równoczesnym przygotowaniu danych wejściowych na CPU w trakcie przetwarzania danych na GPU.

<br>Porównanie jakości pracy modeli:

<table>
<thead>
<tr>
<th>Version</th>
<th>Top-1 Accuracy</th>
<th>Top-5 Accuracy</th>
</tr>
</thead>

<tbody>
<tr>
<td>MobileNet V1</td>
<td>70.9</td>
<td>89.9</td>
</tr>

<tr>
<td>MobileNet V2</td>
<td>71.8</td>
<td>91.0</td>
</tr>
</tbody>
</table>


## WNIOSKI

**MobileNet V2 ma lepsze wynik od MobileNet V1 dla wszyskich wymienionych miar. Zarówno jest szybszy jak również ma lepszą dokładaność dla klasyfikacji.**
Wykazano, że MobileNet może być z powodzeniem stosowany do detekcji obiektów i segmentacji obrazów.

<br>Dla klasyfikacji ostatni blok wygląda jak na rysunku poniżej:

<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_Classifier.png?raw=1" alt="Classifier" width="600" >
</div>


<br>Użycie MobileNet w SSDlite do detekcji, może wyglądać następująco:


<div align="center"><br>
<img src="https://github.com/DarekGit/FACES_DNN/blob/master/Figures/MN2/MN2_FeatureExtractor.png?raw=1" alt="Feature Extractor" width="600" ><br>
</div>

<br>W powyższym rozwiązaniu wykorzystano nie tylko cechy z ostatniej warstwy MobileNet, ale również kilka wcześniejszych. Na pokazanym przykładzie MobileNet wykorzystywany jest jako ekstraktor cech obrazu.<br><br><br>


**W dalszej części pracy zastosowano MobileNet V2 w detekcji twarzy w modelu Detectron2 z bardzo dobrym wynikiem.**


<br>

In [ ]:
import torch
from torch import nn
from torch.nn import BatchNorm2d
from detectron2.layers import Conv2d, FrozenBatchNorm2d, ShapeSpec
from detectron2.modeling.backbone.build import BACKBONE_REGISTRY
from detectron2.modeling.backbone import Backbone
from detectron2.modeling.backbone.fpn import FPN, LastLevelMaxPool

from detectron2.layers import get_norm

class Mish(nn.Module):
    def __init__(self):
        super().__init__()
  
    def forward(self, x):
        x = x * (torch.tanh(nn.functional.softplus(x)))
        return x



def conv_bn(inp, oup, stride,norm='BN',ReL=True):
    return nn.Sequential(
        Conv2d(inp, oup, 3, stride, 1, bias=False),
        get_norm(norm,oup), #zamiast FrozenBatchNorm2d(oup),
        nn.ReLU6(inplace=True) if ReL else Mish()
    )


def conv_1x1_bn(inp, oup,norm='BN',ReL=True):
    return nn.Sequential(
        Conv2d(inp, oup, 1, 1, 0, bias=False),
        get_norm(norm,oup),
        nn.ReLU6(inplace=True) if ReL else Mish()
    )


class InvertedResidual(nn.Module):
    def __init__(self, inp, oup, stride, expand_ratio,norm='BN',ReL=True):
        super(InvertedResidual, self).__init__()
        self.stride = stride
        assert stride in [1, 2]

        hidden_dim = int(round(inp * expand_ratio))
        self.use_res_connect = self.stride == 1 and inp == oup

        if expand_ratio == 1:
            self.conv = nn.Sequential(
                # dw
                Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                get_norm(norm,hidden_dim),
                nn.ReLU6(inplace=True) if ReL else Mish(),
                # pw-linear
                Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                get_norm(norm,oup),
            )
        else:
            self.conv = nn.Sequential(
                # pw
                Conv2d(inp, hidden_dim, 1, 1, 0, bias=False),
                get_norm(norm,hidden_dim),
                nn.ReLU6(inplace=True) if ReL else Mish(),
                # dw
                Conv2d(hidden_dim, hidden_dim, 3, stride, 1, groups=hidden_dim, bias=False),
                get_norm(norm,hidden_dim),
                nn.ReLU6(inplace=True) if ReL else Mish(),
                # pw-linear
                Conv2d(hidden_dim, oup, 1, 1, 0, bias=False),
                get_norm(norm,oup),
            )

    def forward(self, x):
        if self.use_res_connect:
            return x + self.conv(x)
        else:
            return self.conv(x)


class MobileNetV2(Backbone):
    """
    Should freeze bn
    """
    def __init__(self, cfg, n_class=1000, input_size=224, width_mult=1.):
        super(MobileNetV2, self).__init__()
        block = InvertedResidual
        input_channel = 32
        norm=cfg.MODEL.MOBILENET.NORM
        ReL=True
        if 'ACTIVATION' in cfg.MODEL.MOBILENET.keys():
          if cfg.MODEL.MOBILENET.ACTIVATION == 'Mish': ReL = False
          
        interverted_residual_setting = [
            # t, c, n, s
            [1, 16, 1, 1],
            [6, 24, 2, 2],
            [6, 32, 3, 2],
            [6, 64, 4, 2],
            [6, 96, 3, 1],
            [6, 160, 3, 2],
            [6, 320, 1, 1],
        ]

        # building first layer
        assert input_size % 32 == 0
        input_channel = int(input_channel * width_mult)
        self.return_features_indices = [3, 6, 13, 17]
        self.return_features_num_channels = []
        self.features = nn.ModuleList([conv_bn(3, input_channel, 2,norm,ReL=ReL)])
        # building inverted residual blocks
        for t, c, n, s in interverted_residual_setting:
            output_channel = int(c * width_mult)
            for i in range(n):
                if i == 0:
                    self.features.append(block(input_channel, output_channel, s, expand_ratio=t,norm=norm,ReL=ReL))
                else:
                    self.features.append(block(input_channel, output_channel, 1, expand_ratio=t,norm=norm,ReL=ReL))
                input_channel = output_channel
                if len(self.features) - 1 in self.return_features_indices:
                    self.return_features_num_channels.append(output_channel)

        self._initialize_weights()
        self._freeze_backbone(cfg.MODEL.BACKBONE.FREEZE_AT)

    def _freeze_backbone(self, freeze_at):
        for layer_index in range(freeze_at):
            for p in self.features[layer_index].parameters():
                p.requires_grad = False

    def forward(self, x):
        res = []
        for i, m in enumerate(self.features):
            x = m(x)
            if i in self.return_features_indices:
                res.append(x)
        return {'res{}'.format(i + 2): r for i, r in enumerate(res)}

    def _initialize_weights(self):
        for m in self.modules():
            if isinstance(m, Conv2d):
                n = m.kernel_size[0] * m.kernel_size[1] * m.out_channels
                m.weight.data.normal_(0, (2. / n) ** 0.5)
                if m.bias is not None:
                    m.bias.data.zero_()
            elif isinstance(m, BatchNorm2d):
                m.weight.data.fill_(1)
                m.bias.data.zero_()
            elif isinstance(m, nn.Linear):
                n = m.weight.size(1)
                m.weight.data.normal_(0, 0.01)
                m.bias.data.zero_()


                

<br>

< [5.1. Detectron2 Faster R-CNN z FPN Resnet50](05_01_DETECTRON2.ipynb) | [5.2. MOBILENETV2](05_02_MOBILENETV2.ipynb) | [5.3. Techniki szybkiego i stabilnego uczenia GSN.](05_03_FrozenBN_Mish.ipynb) >

---

[Spis treści](https://github.com/DarekGit/FACES_DNN/blob/master/notebooks/Praca_Dyplomowa.ipynb) | [1. Wstęp](01_00_Wstep.ipynb) | [2. Metryki oceny detekcji](02_00_Miary.ipynb) | [3. Bazy danych](03_00_Datasety.ipynb) | [4. Przegląd metod detekcji](04_00_Modele.ipynb) | [5. Detekcja twarzy z wykorzystaniem wybranych architektur GSN](05_00_Modyfikacje.ipynb) | [6. Porównanie modeli](06_00_Porownanie.ipynb) | [7. Eksport modelu](07_00_Eksport_modelu.ipynb) | [8. Podsumowanie i wnioski](08_00_Podsumowanie.ipynb) | [Bibliografia](Bibliografia.ipynb)


---
